### Retrieve all the exercise urls and Images

In [ ]:

import requests 
from bs4 import BeautifulSoup 
import json
import time

domain = "https://www.rehabhero.ca"
url_main = "https://www.rehabhero.ca/exercise"

urls = []
final = True
next_url = url_main

data = []


while final:
    html_doc = requests.get(next_url)
    while html_doc.status_code == 429: # overcome requests number limitation 
        print("sleeping")
        time.sleep(2)
        html_doc = requests.get(next_url)

    print("scrapping")
    soup = BeautifulSoup(html_doc.text, 'html.parser')
    div_contianer = soup.find("div", class_="blog-basic-grid collection-content-wrapper")
    for article in div_contianer.findAll("article"):
        item = {}
        item["url"] = domain + article.div.a["href"]
        item["img"] = article.div.a.img["data-src"]
        data.append(item)
    next_anchor = div_contianer.find("div", class_="older")

    if next_anchor.a:
        next_url = domain + next_anchor.a["href"]
    else:
        final = False



### For each exercise url, retrieve the exercise details

In [ ]:
import copy
data_ = copy.deepcopy(data) # for safety

In [ ]:
for i in range(len(data_)):
    item = data_[i]
    html_doc = requests.get(item["url"])
    while html_doc.status_code == 429: # overcome requests number limitation 
        print("sleeping")
        time.sleep(2)
        html_doc = requests.get(item["url"])

    soup = BeautifulSoup(html_doc.text, 'html.parser')
    div_container = soup.find("div","blog-item-inner-wrapper")
    data_[i]["title"] = div_container.find("h1").text.strip()
    div_video = div_container.find("div","sqs-video-wrapper")
    soup_video = BeautifulSoup(div_video["data-html"], 'html.parser')
    video_url = soup_video.find("iframe")["src"]
    data_[i]["video_url"] = video_url.strip("//")

    paragraphs =  div_container.find_all("p")[1:]
    label = "desc"
    data_[i][label] = []
    for p in paragraphs:
        if p.strong:
            label = p.strong.text.lower().replace(" ","_").strip().strip(":")
            data_[i][label] = []
        else:
            data_[i][label].append(p.text.strip())
    
    data_[i]["desc"] = " ".join(data_[i]["desc"])
    
    if "required_exercise_equipment" in data_[i]: # retrieve exercise equipment
        tmp = []
        for e in data_[i]["required_exercise_equipment"]:
            e = e.lower()
            e = e.replace("click to purchase","")
            e = e.replace("-","")
            e = e.replace("click to buy","")
            e = e.strip()
            if e != "":
                tmp.append(e)
        if tmp:
            data_[i]["required_exercise_equipment"] = tmp
        else:
            del data_[i]["required_exercise_equipment"]

    if "exercise_equipment" in data_[i]: # retrieve exercise equipment
        tmp = []
        for e in data_[i]["exercise_equipment"]:
            e = e.lower()
            e = e.replace("click to purchase","")
            e = e.replace("-","")
            e = e.replace("click to buy","")
            e = e.strip()
            if e != "":
                tmp.append(e)
        if tmp:
            data_[i]["exercise_equipment"] = tmp
        else:
            del data_[i]["exercise_equipment"]

    


### Parse and store the data

In [ ]:
with open("out.json","w") as out_f:
    json.dump(data_,out_f,indent=4,ensure_ascii=False)
    out_f.flush()


In [ ]:
import json
exercises = []
with open("exercises.json") as in_f:
    exercises = json.load(in_f)
    conditions = {}
    for e in exercises:
        if "related_conditions" in e :
            for c in e["related_conditions"]:
                if c in conditions:
                    conditions[c].append(e)
                else:
                    conditions[c] = [e]

res = []
for k,v in conditions.items():
    res.append({"condition":k, "exercises":v})

with open("conditionss.json","w") as out_f:
    json.dump(res,out_f,indent=4,ensure_ascii=False)
    out_f.flush()

In [ ]:
len(conditions)